# Recomendador de artigos - Open Alex

## Importação das bibliotecas

In [1]:
import pandas as pd
import os
import requests


pd.options.display.max_columns = 999

## Funções

In [2]:
def filtrar_dataframe_por_termos(df, termos):
    
    termos_formatados = [f"`{termo}`" if ' ' in termo else termo for termo in termos]
    
    filtro = " or ".join(f"{termo} > 0" for termo in termos_formatados)
    
    df_filtrado = df.query(filtro).loc[:, ['doi', 'title','abstract', 'publication_date', 'open_access'] + termos]
    
    return df_filtrado

def filtrar_dataframe_por_acesso_aberto(df,resposta):
    
    if resposta == 'Sim':
        df = df.query('open_access == True')
    
    return df

def criar_coluna_score(df):
    
    df['score'] = df.iloc[:,4:].sum(axis=1)
    
    return df

def atribuir_fator_termo_score(df, termos):
    df_copy = df.copy()
    
    df_copy['title'] = df_copy['title'].fillna('')
    
    for term in termos:
        mask = df_copy['title'].str.upper().str.contains(term.upper())
        df_copy.loc[mask, 'score'] *= 2
        
        mask = df_copy['abstract'].str.upper().str.contains(term.upper())
        df_copy.loc[mask, 'score'] *= 1.5
    
    df_copy = df_copy.sort_values(by='score', ascending=False)
    
    return df_copy

def atribuir_fator_termo_similar_score(df, termos_similares):
    df_copy = df.copy()
    
    df_copy['title'] = df_copy['title'].fillna('')
    
    for term in termos_similares:
        mask = df_copy['title'].str.upper().str.contains(term.upper())
        df_copy.loc[mask, 'score'] *= 1.5
        
        mask = df_copy['abstract'].str.upper().str.contains(term.upper())
        df_copy.loc[mask, 'score'] *= 1.25
    
    df_copy = df_copy.sort_values(by='score', ascending=False)
    
    return df_copy


def normalizar_score(df):
    min_score = df['score'].min()
    max_score = df['score'].max()


    df['score_normalizado'] = ((df['score'] - min_score) / (max_score - min_score))

    return df

## Verificação dos dados

In [3]:
df = pd.read_parquet('../data/processed/df_concatenado.parquet')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59437 entries, 0 to 59436
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   doi                    59437 non-null  object 
 1   title                  59437 non-null  object 
 2   abstract               59437 non-null  object 
 3   publication_date       59437 non-null  object 
 4   open_access            59437 non-null  bool   
 5   concepts               59437 non-null  object 
 6   Computer science       59437 non-null  float64
 7   Mathematics            59437 non-null  float64
 8   Physics                59437 non-null  float64
 9   Biology                59437 non-null  float64
 10  Chemistry              59437 non-null  float64
 11  Political science      59437 non-null  float64
 12  Engineering            59437 non-null  float64
 13  Materials science      59437 non-null  float64
 14  Philosophy             59437 non-null  float64
 15  Bu

In [6]:
df.query('abstract == ""').shape[0]

10851

In [7]:
df.query('abstract == ""').shape[0] / df.shape[0]

0.18256304995205008

In [96]:
concepts = []
qtd_de_valores_nao_nulos = []
pct_valores_nao_nulos = []

for i,j in df.iloc[:,6:].items():
    concepts.append(i)
    qtd_de_valores_nao_nulos.append(df.shape[0] - sum(j==0))
    pct_valores_nao_nulos.append(round(((df.shape[0] - sum(j==0))/ df.shape[0]),2))
    
    
pd.DataFrame({'concepts':concepts,
              'nao_nulos':qtd_de_valores_nao_nulos,
              'percentual':pct_valores_nao_nulos}).sort_values(by='percentual',ascending=False)

,concepts,nao_nulos,percentual
12,Medicine,19176,0.32
0,Computer science,17706,0.30
3,Biology,11332,0.19
2,Physics,8156,0.14
4,Chemistry,8315,0.14
10,Psychology,8282,0.14
6,Engineering,7170,0.12
7,Materials science,6878,0.12
1,Mathematics,7093,0.12
5,Political science,6382,0.11


In [28]:
for i in df.columns:
    print(i)

doi
title
abstract
publication_date
open_access
concepts
Computer science
Mathematics
Physics
Biology
Chemistry
Political science
Engineering
Materials science
Philosophy
Business
Psychology
Art
Medicine
Geography
Geology
Economics
Sociology
Environmental science
History


In [9]:
df_filtrado = filtrar_dataframe_por_termos(df,['Economics'])

df_filtrado = filtrar_dataframe_por_acesso_aberto(df_filtrado,'Não')

df_filtrado = criar_coluna_score(df_filtrado)

df_filtrado = atribuir_fator_termo_score(df_filtrado,['Econometria'])

df_filtrado = atribuir_fator_termo_similar_score(df_filtrado,
                                                 ['Econometrics', 'Regression analysis', 
                                                  'Time series analysis', 'Statistical modeling',
                                                  'Hypothesis testing','Econometria', 'Análise de regressão',
                                                  'Análise de séries temporais', 'Modelagem estatística',
                                                  'Teste de hipótese'])

df_filtrado = normalizar_score(df_filtrado)

In [12]:
def filtrar_escolha(areas,acesso_aberto,termo,termo_similar):
    
    df_filtrado = filtrar_dataframe_por_termos(df,areas)

    df_filtrado = filtrar_dataframe_por_acesso_aberto(df_filtrado,acesso_aberto)

    df_filtrado = criar_coluna_score(df_filtrado)

    df_filtrado = atribuir_fator_termo_score(df_filtrado,termo)

    df_filtrado = atribuir_fator_termo_similar_score(df_filtrado,termo_similar)

    df_filtrado = normalizar_score(df_filtrado)

    return df_filtrado.head(10)

In [13]:
filtrar_escolha(['Economics'],'Não',['Econometria'],['Econometrics', 'Regression analysis', 
                                                  'Time series analysis', 'Statistical modeling',
                                                  'Hypothesis testing','Econometria', 'Análise de regressão',
                                                  'Análise de séries temporais', 'Modelagem estatística',
                                                  'Teste de hipótese'])


,doi,title,abstract,publication_date,open_access,Economics,score,score_normalizado
37754,https://doi.org/10.21203/rs.3.rs-3376767/v1,Fiscal space and government-spending cyclicali...,Abstract This study employs quantile regressio...,2023-10-06,True,0.6334,2.041750,1.000000
45358,https://doi.org/10.31153/js.v25i2.992,CONTRIBUTION OF INDONESIAN NATIONAL STANDARD (...,&lt;p&gt;Gross Domestic Product (GDP) is the n...,2023-10-04,True,0.6208,2.026000,0.992155
7177,https://doi.org/10.31092/jpkn.v5i1.2315,The Effect of Per Capita Income and the Agricu...,The purpose of this research is to analyze the...,2023-10-03,True,0.6049,2.006125,0.982254
51326,https://doi.org/10.36713/epra14543,AN ANALYTICAL STUDY OF PUBLIC HEALTH EXPENDITU...,This study examines a comparative analysis of ...,2023-10-05,True,0.5467,1.933375,0.946016
36425,https://doi.org/10.29235/1818-9806-2023-9-3-22,Regression analysis of the efficiency of the u...,The author’s methodology of analysis has been ...,2023-10-03,True,0.2649,1.897350,0.928072
45293,https://doi.org/10.33395/owner.v7i4.1937,The Influence of Original Local Government Rev...,This study aims to examine how the influence o...,2023-10-02,True,0.5055,1.881875,0.920363
13246,https://doi.org/10.4038/ija.v3i1.52,Factors Affecting Share Prices of Finance Comp...,The price of a share is influenced by The vari...,2023-10-05,True,0.4991,1.873875,0.916378
26525,https://doi.org/10.24912/jm.v27i3.1300,The Most Influential Factors On Stock Prices I...,This study determined the impact of changes in...,2023-10-04,True,0.4828,1.853500,0.906229
22933,https://doi.org/10.9734/jemt/2023/v29i101153,Determinant of Educated Unemployment Rate in W...,This research was conducted to find out how th...,2023-10-02,True,0.4535,1.816875,0.887985
20276,https://doi.org/10.11611/yead.1316839,THE INFLUENCE OF FISCAL POLICY ON UNEMPLOYMENT...,"In the context of ongoing economic discussion,...",2023-10-04,True,0.8063,1.806300,0.882718


In [14]:
df_filtrado_top = df_filtrado.head(5).reset_index(drop=True).loc[:,['doi','title','abstract','publication_date']]

for i in range(5):
    print(f"'TÍTULO': {df_filtrado_top['title'].iloc[i]}")
    print(f"'RESUMO': {df_filtrado_top['abstract'].iloc[i]}")
    print(f"'DATA DE PUBLICAÇÃO': {df_filtrado_top['publication_date'].iloc[i]}")
    print(f"'DOI': {df_filtrado_top['doi'].iloc[i]}")    
    print(100*'-')

'TÍTULO': Fiscal space and government-spending cyclicality: Disparity between the procyclical and the countercyclical
'RESUMO': Abstract This study employs quantile regression analysis on a dataset encompassing 160 countries spanning the period from 1990 to 2020. Its primary objective is study investigate analysis relationship between fiscal space and various conditional quantiles of government-spending cyclicality. Unlike prior literature, which predominantly centers on government debt sustainability, our on introduces a comprehensive perspective to encompassing space, the dimensions that have received comparatively limited attention. These the include sovereign balance sheet vulnerability, contingent liabilities arising the risks associated with external the private sector debt, the market perceptions the the risk. Our the suggests dimensions from to is statistically significant only at fiscal upper part fiscal space and and cyclicality distribution, i.e., fiscally procyclical countr

In [34]:
with open('/home/franciscofoz/Documents/GitHub/recomendador-artigos-OpenAlex-GPT/newsletter.html','r', encoding='utf-8') as file:
    html_template = file.read()
    
html_final = ""

for i in range(5):
    article_html = f"""
        <div class="article">
            <h2> {df_filtrado_top['title'].iloc[i]}</h2>
            <p> {df_filtrado_top['abstract'].iloc[i]}</p>
            <p class="date"> Data de Publicação: {df_filtrado_top['publication_date'].iloc[i]}</p>
            <p class="doi">DOI: <a href="{df_filtrado_top['doi'].iloc[i]}">{df_filtrado_top['doi'].iloc[i]}</a></p>
        </div>
    """


    html_final += article_html

html_template = html_template.replace("<!-- ARTIGOS_AQUI -->", html_final)

with open('../template_final.html', 'w', encoding='utf-8') as file:
    file.write(html_template)

#PROMPT:

'''

A partir desse termo ['Econometria']. 

Gere 5 termos relacionados em inglês e suas respectivas traduções em português. 

Responda com uma única lista Python.

Como nesse exemplo: 
['"Artificial Intelligence","Inteligência Artificial"]

Não responda mais nada além da lista.

'''

'''
Sou uma biblioteca que gostaria de recomendar novos papers para seus usuários. 

A partir desses papers publicados na última semana:

[]

Crie um e-mail recomendando cada um desses papers, como uma newletter científica promovendo cada um dos papers.

Insira um parágrafo inicial no um e-mail, com um resumo do conteúdo.

NÃO TRADUZA OS TÍTULOS.

Este é um e-mail pessoal para ['Francisco Foz']

Responda sempre:
- TÍTULO:
- RESUMO:
- DATA DE PUBLICAÇÃO:
- DOI:
'''

Faça 